In [1]:
import pandas as pd
from pathlib import Path

In [14]:
root = Path('/media/ts/SSD_ubuntu/datasets/')

audioset_train = pd.read_csv(root / 'AudioSet/audioset_train.csv')
audioset_tag2name = pd.read_csv(root / 'AudioSet/class_labels_indices.csv').set_index('mid')
audioset_tag2name = audioset_tag2name['display_name'].to_dict()

In [58]:
for split in ['train', 'test', 'val']:
    df = pd.read_parquet(root / f'AudioCaps/annot_{split}.parquet')
    df['youtube_id'] = df.filename.apply(lambda x: Path(x).stem[:11])
    df['time_start'] = df.filename.apply(lambda x: float(Path(x).stem[12:18]))
    df['time_end'] = df.filename.apply(lambda x: float(Path(x).stem[19:]))
    # merging over 'youtube_id' (excluding 'time_start', 'time_end') is sufficient since the
    # id is unique and AudioCaps round the clips to 10s.
    merge = df.merge(audioset_train, on=['youtube_id'], how='inner')
    assert len(merge) == len(df)
    merge['tags'] = merge.positive_labels.apply(lambda x: [audioset_tag2name[i] for i in x.strip('"').split(',')])
    merge = merge.drop(columns=['time_start_x', 'time_end_x', 'time_end_y', 'time_start_y', 'youtube_id', 'positive_labels'])
    merge.to_parquet(root / f'AudioCaps/annot_tags_{split}.parquet', index=False)

In [3]:
root = Path('/media/t9s9/SSD_ubuntu/datasets/')
pd.read_parquet(root /f'AudioCaps/annot_tags_train.parquet' )

,filename,captions,tags
0,train/---1_cCGK4M_000000_000010.mp4,"[Idling car, train blows horn and passes]","[Railroad car, train wagon, Train horn, Rail t..."
1,train/---lTs1dxhU_000030_000040.mp4,[A racing vehicle engine is heard passing by],"[Motor vehicle (road), Vehicle, Car, Car passi..."
2,train/--0PQM4-hqg_000030_000040.mp4,[A waterfall is pouring into a stream],"[Gurgling, Waterfall, Stream]"
3,train/--1l6WGE7AU_000030_000040.mp4,[A man speaks earnestly],"[Narration, monologue, Male speech, man speaki..."
4,train/--299m5_DdE_000000_000010.mp4,[Water flows as a child shouts in the background],"[Gurgling, Waterfall]"
...,...,...,...
45794,train/zzlfP-snUeY_000010_000020.mp4,[A vehicle engine is idling],"[Engine, Idling, Vehicle, Outside, rural or na..."
45795,train/zzm3dwoXY8Y_000030_000040.mp4,[A man talking as a flock of pigeons coo follo...,"[Bird, Speech, Pigeon, dove, Outside, rural or..."
45796,train/zzvWbSyZfr0_000000_000010.mp4,[A person snoring as a man speaks through a te...,"[Snoring, Speech]"
45797,train/zzwBazlj0Oc_000030_000040.mp4,[Several pigeons coo and flap their wings],"[Bird, Pigeon, dove, Inside, small room]"


In [4]:
pd.read_parquet('/media/t9s9/SSD_ubuntu/datasets/AudioCaps/visual_captions/visual_captions_train.parquet')

,captions
filename,
3VgonYwfqwo_000170_000180,how to build a cnc machine from the very begin...
nLxbwx-eu9w_000020_000030,herd of goats on a farm.
3xiiXWQA8cY_000030_000040,friends having fun at night in the living room.
YFZEH0SYrCU_000120_000130,this is the first time i've seen this type of ...
HlJdfKYg6As_000030_000040,a close up of a clock attached to a wall ...
...,...
0kBc1q-dm6I_000030_000040,children ride on a roller coaster.
77W8716C44A_000030_000040,A large green frog floating in the water.
TFydYI3enJM_000030_000040,A woman sitting on a pink wall talking on a ce...
